# Webscraping Traditional NBA Pipeline

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season='
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)[-2:] + "&SeasonType=Regular%20Season"
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2021-22&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2020-21&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2019-20&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2018-19&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2017-18&SeasonType=Regular%20Season']

In [4]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:
    # provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Click on accept cookies button
    driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button").click()

    # Make python pause for some time
    sleep(4)

    # Select table page
    select_page = Select(driver.find_element(By.XPATH,"/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select"))
    select_page.select_by_visible_text('All')

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('th'):
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Remove redundant headers
    headers_2 = ['PLAYER','TEAM','AGE','GP','W','L','MIN','PTS','FGM','FGA',
                 'FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB', 
                 'REB','AST','TOV','STL','BLK','PF','FP','DD2','TD3','+/-']

    # Create a dataframe using the headers as column names
    nba_stat = pd.DataFrame(columns = headers_2) 

    # Create a for loop to fill nba_stat
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "") for j in row_data] 
        length = len(nba_stat)
        nba_stat.loc[length] = row
    
    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(nba_stat)

Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 24 webpage(s) scraped.
Completed: 25

In [5]:
# dataframe_list is a list of all dataframes
dataframe_list

[             PLAYER TEAM AGE  GP   W   L   MIN   PTS              FGM  \
 0        Trae Young  ATL  23  76  40  36  2652  2155              711   
 1     DeMar DeRozan  CHI  32  76  43  33  2743  2118              774   
 2       Joel Embiid  PHI  28  68  45  23  2296  2079              666   
 3      Jayson Tatum  BOS  24  76  49  27  2731  2046              708   
 4      Nikola Jokic  DEN  27  74  46  28  2476  2004              764   
 ..              ...  ...  ..  ..  ..  ..   ...   ...              ...   
 600     Nate Hinton  IND  23   2   0   2     2     0                0   
 601      Sam Dekker  TOR  28   1   1   0     1     0                0   
 602  Trayvon Palmer  DET  27   1   0   1    17     0                0   
 603      Tyler Hall  NYK  25   1   1   0     2     0                0   
 604  Zylan Cheatham  UTA  26   1   0   1     5     0                0   
 
                   FGA  ...               REB              AST  \
 0                1544  ...               28

In [6]:
# Concat all the dataframes into one table
nba_traditional = pd.concat(dataframe_list, ignore_index=True)
nba_traditional

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Trae Young,ATL,23,76,40,36,2652,2155,711,1544,...,284,737,303,72,7,128,3535.3,42,0,159
1,DeMar DeRozan,CHI,32,76,43,33,2743,2118,774,1535,...,392,374,181,68,24,178,3244.4,6,0,77
2,Joel Embiid,PHI,28,68,45,23,2296,2079,666,1334,...,796,284,214,77,99,181,3774.2,46,2,368
3,Jayson Tatum,BOS,24,76,49,27,2731,2046,708,1564,...,609,334,217,75,49,174,3432.8,22,0,667
4,Nikola Jokic,DEN,27,74,46,28,2476,2004,764,1311,...,1019,584,281,109,63,191,4337.8,66,19,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,ATL,25,1,0,1,14,0,0,5,...,7,0,0,0,0,2,8.4,0,0,-14
12303,Bruce Bowen,MIA,26,1,1,0,1,0,0,0,...,0,0,0,0,1,0,3.0,0,0,3
12304,Cuonzo Martin,MIL,25,3,0,3,13,0,0,7,...,1,1,1,0,0,1,1.7,0,0,-2
12305,Derrick Alston,ATL,24,2,1,1,11,0,0,5,...,4,0,0,0,0,0,4.8,0,0,-7


In [15]:
# Check datatypes
nba_traditional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PLAYER  12307 non-null  object
 1   TEAM    12307 non-null  object
 2   AGE     12307 non-null  object
 3   GP      12307 non-null  object
 4   W       12307 non-null  object
 5   L       12307 non-null  object
 6   MIN     12307 non-null  object
 7   PTS     12307 non-null  object
 8   FGM     12307 non-null  object
 9   FGA     12307 non-null  object
 10  FG%     12307 non-null  object
 11  3PM     12307 non-null  object
 12  3PA     12307 non-null  object
 13  3P%     12307 non-null  object
 14  FTM     12307 non-null  object
 15  FTA     12307 non-null  object
 16  FT%     12307 non-null  object
 17  OREB    12307 non-null  object
 18  DREB    12307 non-null  object
 19  REB     12307 non-null  object
 20  AST     12307 non-null  object
 21  TOV     12307 non-null  object
 22  STL     12307 non-null

In [16]:
# Get column names
nba_traditional.columns

Index(['PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-'],
      dtype='object')

In [17]:
# Change datatypes to numeric for appropriate columns
nba_traditional[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
                 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 
                 'OREB', 'DREB', 'REB','AST', 'TOV', 'STL', 'BLK', 
                 'PF', 'FP', 'DD2', 'TD3', '+/-']] = nba_traditional[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 
                                                                      'FGM', 'FGA','FG%', '3PM', '3PA', '3P%', 
                                                                      'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
                                                                      'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 
                                                                      'TD3', '+/-']].apply(pd.to_numeric)

In [18]:
# Verify datatype changes
nba_traditional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  12307 non-null  object 
 1   TEAM    12307 non-null  object 
 2   AGE     12307 non-null  int64  
 3   GP      12307 non-null  int64  
 4   W       12307 non-null  int64  
 5   L       12307 non-null  int64  
 6   MIN     12307 non-null  int64  
 7   PTS     12307 non-null  int64  
 8   FGM     12307 non-null  int64  
 9   FGA     12307 non-null  int64  
 10  FG%     12307 non-null  float64
 11  3PM     12307 non-null  int64  
 12  3PA     12307 non-null  int64  
 13  3P%     12307 non-null  float64
 14  FTM     12307 non-null  int64  
 15  FTA     12307 non-null  int64  
 16  FT%     12307 non-null  float64
 17  OREB    12307 non-null  int64  
 18  DREB    12307 non-null  int64  
 19  REB     12307 non-null  int64  
 20  AST     12307 non-null  int64  
 21  TOV     12307 non-null  int64  
 22

In [10]:
player = nba_traditional[nba_traditional['PLAYER'] =='Stephen Curry']
player

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
13,Stephen Curry,GSW,34,64,45,19,2211,1630,535,1224,...,335,404,206,85,23,130,2756.0,12,2,509
605,Stephen Curry,GSW,33,63,37,26,2152,2015,658,1365,...,345,363,213,77,8,119,3015.5,8,0,244
1529,Stephen Curry,GSW,32,5,1,4,139,104,33,82,...,26,33,16,5,2,11,189.7,1,0,-45
1681,Stephen Curry,GSW,31,69,52,17,2331,1881,632,1340,...,369,361,192,92,25,166,3024.3,3,0,689
2233,Stephen Curry,GSW,30,51,41,10,1631,1346,428,864,...,261,310,153,80,8,114,2235.2,5,0,486
2751,Stephen Curry,GSW,29,79,65,14,2639,1999,675,1443,...,353,524,239,142,17,183,3446.6,9,0,1015
3231,Stephen Curry,GSW,28,79,71,8,2700,2375,805,1598,...,430,527,262,169,15,161,3971.5,15,2,1022
3707,Stephen Curry,GSW,27,80,67,13,2613,1900,653,1341,...,341,619,249,163,16,158,3525.7,23,0,919
4203,Stephen Curry,GSW,26,78,50,28,2846,1873,652,1383,...,334,666,294,128,14,194,3404.8,28,4,574
4686,Stephen Curry,GSW,25,78,45,33,2983,1786,626,1388,...,314,539,240,126,12,198,3145.3,15,0,133


In [11]:
curry = player[player['AGE'] == '22']
curry

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
6107,Stephen Curry,GSW,22,80,25,55,2896,1399,528,1143,...,356,472,243,152,19,252,2804.2,12,1,-187


In [14]:
curry.to_csv('curry.csv', encoding='utf-8')

In [8]:
# Save as csv file
nba_traditional_csv = nba_traditional.to_csv('nba_traditional_final.csv', index=False)

In [9]:
# Check that it's saved
df_nba = pd.read_csv('nba_traditional_final.csv')
df_nba

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Trae Young,ATL,23,76,40,36,2652,2155,711,1544,...,284,737,303,72,7,128,3535.3,42,0,159
1,DeMar DeRozan,CHI,32,76,43,33,2743,2118,774,1535,...,392,374,181,68,24,178,3244.4,6,0,77
2,Joel Embiid,PHI,28,68,45,23,2296,2079,666,1334,...,796,284,214,77,99,181,3774.2,46,2,368
3,Jayson Tatum,BOS,24,76,49,27,2731,2046,708,1564,...,609,334,217,75,49,174,3432.8,22,0,667
4,Nikola Jokic,DEN,27,74,46,28,2476,2004,764,1311,...,1019,584,281,109,63,191,4337.8,66,19,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,ATL,25,1,0,1,14,0,0,5,...,7,0,0,0,0,2,8.4,0,0,-14
12303,Bruce Bowen,MIA,26,1,1,0,1,0,0,0,...,0,0,0,0,1,0,3.0,0,0,3
12304,Cuonzo Martin,MIL,25,3,0,3,13,0,0,7,...,1,1,1,0,0,1,1.7,0,0,-2
12305,Derrick Alston,ATL,24,2,1,1,11,0,0,5,...,4,0,0,0,0,0,4.8,0,0,-7


# Prepare and Clean Data

In [ ]:
# Look at dimension
nba_traditional.shape

In [ ]:
# Remove columns that are redundant, unnecessary
nba_traditional = nba_traditional.drop(['TEAM','AGE', 'FGM','FGA','3PM','3PA',
                          'FTM','FTA','OREB','DREB','TOV','PF','FP',
                          'DD2','TD3','+/-'], axis=1)

nba_traditional.head()

In [ ]:
# Look at dimension
nba_traditional.shape

In [ ]:
# Find duplicate rows, if any
duplicateRows = nba_traditional[nba_traditional.duplicated()]
duplicateRows

In [ ]:
# Find null counts
null_counts = nba_traditional.isna().sum()
null_counts

In [ ]:
# Check datatypes
nba_traditional.info()

In [ ]:
nba_traditional.columns

In [ ]:
# Change object datatypes to numeric
nba_traditional[['GP', 'W', 'L', 'PTS', 'FG%', '3P%', 'FT%', 'REB', 'AST',
       'STL', 'BLK']] = nba_traditional[['GP', 'W', 'L', 'PTS', 'FG%', '3P%', 'FT%', 'REB', 'AST',
       'STL', 'BLK']].apply(pd.to_numeric)

In [ ]:
# Confirm datatype changes
nba_traditional.info()

## Save into csv file

In [ ]:
nba_traditional_csv = nba_traditional.to_csv('nba_traditional.csv', index=False)

In [ ]:
# Check that it's saved
df_nba = pd.read_csv('nba_traditional.csv')
df_nba

In [ ]:
# Remove columns that are noncontritory
df_nba = df_nba.drop(['REB', 'AST','STL', 'BLK', 'GP', 'W', 'L'], axis=1)
df_nba

In [ ]:
# Which row in column 'PLAYER' has duplicates 
# (i.e. same player can show up in multiple seasons with different stats per season)
duplicaterows = df_nba[df_nba.duplicated(['PLAYER'])]
duplicaterows

In [ ]:
# Groupby the player column, some players show up in multiple seasons, average out the stats
df_nba_avg = df_nba.groupby('PLAYER').mean(['PTS', 'FG%', '3P%', 'FT%'])
df_nba_avg.head()

In [ ]:
# Save the dataframe
df_nba_avg_csv = df_nba_avg.to_csv('nba_avg.csv')

In [ ]:
# Check that it's saved
df_nba_avg = pd.read_csv('nba_avg.csv')
df_nba_avg